###### AOS_ts_phosim_tests

Inspect each test, to see what needs to be changed. 

I make a ticket branch https://jira.lsstcorp.org/browse/DM-29233 

    
    cd /project/scichris/aos/ts_phosim/
    git checkout -b tickets/DM-29233 
    

Need to make sure that the jupyter kernel has the appropriate setup. Add the following to `/home/scichris/notebooks/.user_setups` : 

    export PATH_TO_TS_PHOSIM=/project/scichris/aos/ts_phosim/
    setup ts_phosim -t $USER -t current 
    
    
 
    

## `test_teleFacade.py`

In [1]:
import os
import shutil
import numpy as np
import unittest

from lsst.ts.wep.Utility import FilterType, CamType

from lsst.ts.phosim.telescope.TeleFacade import TeleFacade

from lsst.ts.phosim.OpdMetrology import OpdMetrology
from lsst.ts.phosim.SkySim import SkySim

from lsst.ts.phosim.Utility import getModulePath, getConfigDir


In [2]:
import tempfile

configDir = getConfigDir()
modulePath = getModulePath()
testDir = os.path.join(modulePath, "tests")

outputDir = tempfile.TemporaryDirectory(dir=testDir)


In [3]:
print(outputDir)

<TemporaryDirectory '/project/scichris/aos/ts_phosim/tests/tmpfc91rcj_'>


In [4]:
#setUpClass(cls):
tele = TeleFacade()
tele.addSubSys(addCam=True, addM1M3=True, addM2=True)

# Set the survey parameters
obsId = 9006000
filterType = FilterType.G
boresight = (0.2, 0.3)
zAngleInDeg = 27.0912
rotAngInDeg = np.rad2deg(-1.2323)
tele.setSurveyParam(
    obsId=obsId,
    filterType=filterType,
    boresight=boresight,
    zAngleInDeg=zAngleInDeg,
    rotAngInDeg=rotAngInDeg,
)

In [5]:
dofInUm = tele.getDofInUm()
print(dofInUm)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [6]:
print(len(dofInUm))

50


In [7]:
print(np.sum(np.abs(dofInUm)))

0.0


# `test_opdMetrology.py`

In [8]:
import os
import numpy as np
import unittest

from lsst.ts.ofc.Utility import InstName

from lsst.ts.phosim.OpdMetrology import OpdMetrology
from lsst.ts.phosim.Utility import getModulePath

In [9]:

testDataDir = os.path.join(
    getModulePath(), "tests", "testData", "testOpdFunc"
)
metr = OpdMetrology()

In [10]:
# def testAddFieldXYbyDeg()
fieldXInDegree = 0.1
fieldYInDegree = 0.2
metr.addFieldXYbyDeg(fieldXInDegree, fieldYInDegree)


In [11]:
fieldX, fieldY = metr.getFieldXY()

In [14]:
len(fieldX)


1

In [15]:
metr.addFieldXYbyDeg(fieldXInDegree, fieldYInDegree)

In [16]:
fieldX, fieldY = metr.getFieldXY()

In [17]:
len(fieldX)


2

In [41]:
#def testAddFieldXYbyCamPos(self):

sensorName = "R22_S11"
xInpixel = 4004
yInPixel = 4096
metr.addFieldXYbyCamPos(sensorName, xInpixel, yInPixel, testDataDir)

In [42]:
fieldX, fieldY = metr.getFieldXY()

In [43]:
fieldX[-1]


0.11122222222222224

In [44]:
fieldY[-1]

0.11377777777777778

In [45]:
2002 * 0.2 / 3600

0.11122222222222224

In [46]:
2048 * 0.2 / 3600

0.11377777777777778

According to https://github.com/lsst-ts/phosim_syseng4/blob/aos/data/lsst/segmentation.txt ,  R22_S11 is $4004$x$4096$   pixels

In [30]:
from lsst.ts.wep.SourceProcessor import SourceProcessor
sourProc = SourceProcessor()
sourProc.config(sensorName=sensorName)

In [32]:
sourProc.sensorFocaPlaneInDeg[sensorName]

(0.0, 0.0)

In [35]:
pixelXc, pixelYc = sourProc.sensorDimList[sensorName]

In [36]:
pixelXc = pixelXc / 2
pixelYc = pixelYc / 2

In [37]:
pixelXc

2002.0

In [38]:
pixelYc

2048.0

In [39]:
pixelToArcsec = sourProc.settingFile.getSetting("pixelToArcsec")

In [40]:
pixelToArcsec

0.2

# `test_closeLoopTask`

testMakeCalibsLSST - needed to change 48 to 24 for corner sensors ... 

In [3]:
import os
import argparse
import tempfile
import unittest

from lsst.ts.wep.Utility import CamType, FilterType
from lsst.ts.ofc.Utility import InstName

from lsst.ts.phosim.CloseLoopTask import CloseLoopTask
from lsst.ts.phosim.Utility import getModulePath


#def testMakeCalibsLSST(self):
closeLoopTask = CloseLoopTask()

rootTestDir = os.path.join(getModulePath(), "tests")
testDir = tempfile.TemporaryDirectory(dir=rootTestDir)

fakeFlatDir = closeLoopTask.makeCalibs(InstName.LSST, testDir.name)
print(fakeFlatDir)
#self.assertTrue(os.path.exists(fakeFlatDir))

files = os.listdir(fakeFlatDir)
print(len(files))
#self.assertEqual(len(files), 24)

ModuleNotFoundError: No module named 'lsst.ts'

In [50]:
#files

# Update phosim_repackager to handle comcam

https://www.askpython.com/python/oops/python-class-constructor-init-function

# `test_phosimCmpt`

In [2]:
import os
import shutil
import numpy as np
import warnings
import unittest

#from lsst.ts.wep.Utility import FilterType, CamType
from lsst.ts.wep.ParamReader import ParamReader
from lsst.ts.wep.ctrlIntf.SensorWavefrontData import SensorWavefrontData
from lsst.ts.wep.ctrlIntf.MapSensorNameAndId import MapSensorNameAndId

from lsst.ts.ofc.Utility import InstName

from lsst.ts.phosim.telescope.TeleFacade import TeleFacade
from lsst.ts.phosim.SkySim import SkySim
from lsst.ts.phosim.OpdMetrology import OpdMetrology
from lsst.ts.phosim.Utility import getModulePath
from lsst.ts.phosim.PhosimCmpt import PhosimCmpt


ModuleNotFoundError: No module named 'lsst.afw.cameraGeom.detector'

In [ ]:
What the files 